In [1]:
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
print(np.__version__)

2.3.0
1.18.5


In [2]:
# divide uma série temporal univariada em amostras
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # encontra o final da amostra
        end_ix = i + n_steps
        # verifica se passou do final
        if end_ix > len(sequence)-1:
            break
        # junta as partes de entrada e saída do padrão
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
df = pd.read_csv('sp_beaches_update.csv', parse_dates=['Date'])
df = df.sort_values(by=['Date'])
df=df.loc[~df['Enterococcus'].isnull()]
#remover a praia do Leste, da cidade de iguape, pois esta praia sumiu por erosão em 2012
#remover a praia MASSAGUAÇU (AV. M. H. CARVALHO), da cidade de CARAGUATATUBA, não possui mais medições em 2021
#remover a praia RIO PEREQUÊ, da cidade de CUBATÃO, até dez de 2020 possuía medições semanais, depois de janeiro de 2021 as medições passaram a ser mensais
#remover as praias da cidade ILHA ANCHIETA pois não exitem mais medições em 2021
df = df.loc[df['Beach']!='DO LESTE'].loc[df['Beach']!='MASSAGUAÇU (AV. M. H. CARVALHO)'].loc[df['Beach']!='RIO PEREQUÊ'].loc[df['City']!='ILHA ANCHIETA']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71443 entries, 0 to 70616
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   City          71443 non-null  object        
 1   Beach         71443 non-null  object        
 2   Date          71443 non-null  datetime64[ns]
 3   Enterococcus  71443 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.7+ MB


In [5]:
# selecionar praias com medições semanal
df_freq = pd.read_csv('frequencia_praias.csv')
df_freq = df_freq.loc[df_freq['Beach']!='DO LESTE'].loc[df_freq['Beach']!='MASSAGUAÇU (AV. M. H. CARVALHO)'].loc[df_freq['Beach']!='RIO PEREQUÊ'].loc[df_freq['City']!='ILHA ANCHIETA']
df_freq = df_freq.loc[df_freq['Frequency']=='MENSAL']
df_freq

,City,Beach,Frequency
0,CARAGUATATUBA,LAGOA AZUL,MENSAL
2,GUARUJÁ,IPORANGA,MENSAL
3,IGUAPE,JURÉIA,MENSAL
4,ILHA COMPRIDA,BALNEÁRIO ADRIANA,MENSAL
5,ILHA COMPRIDA,CENTRO,MENSAL
6,ILHA COMPRIDA,PONTAL,MENSAL
7,ILHA COMPRIDA,PRAINHA (BALSA),MENSAL
8,UBATUBA,LAGOA PRUMIRIM,MENSAL
9,UBATUBA,PRUMIRIM,MENSAL
10,UBATUBA,PULSO,MENSAL


In [ ]:
n_steps = 4 #4 medições representa 4 meses que serão utilizados para prever a próxima medição
#Para cada cidade 
for cidade in df_freq.City.unique():
    # Para cada praia desta cidade
    for praia in df_freq.loc[df_freq['City']==cidade].Beach.unique():
        #Imprime que iniicou o processo de carregamento de dados de uma determinada praia
        print(f'iniciando: {cidade} - {praia}')
        #Coleta os dados desta praia
        df_beach = df.loc[df['City']==cidade].loc[df['Beach']==praia][['Date','Enterococcus']]
        #formata os dados desta praia para enviar ao modelo
        df_beach.columns = ['ds', 'y']
        df_beach.set_index('ds', inplace=True)
        #divide os dados da praia em lotes de X com n_steps medições para y com 1 medição de rótulo
        X, y = split_sequence(df_beach.values, n_steps)
        #formata de acordo com a entrada do modelo
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))
        #prepara as camadas do modelo
        model = Sequential()
        model.add(LSTM(104, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(LSTM(104, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        #imprime que o modelo irá começar a ser treinado para esta praia
        print('Treinando modelo: '+cidade+'-'+praia)
        #começa o treino do modelo na GPU
        with tf.device('/GPU:0'): 
            R = model.fit(X, y, epochs=200, verbose=0)
        #depois de treinado pega o erro do MSE
        loss=round(R.history['loss'][-1])
        #salva o modelo com o nome de modelcidade-praia.h5
        model.save('model'+cidade+'-'+praia+'.h5')
        #imprime que o modelo foi treinado e o valor do erro MSE
        print('Modelo Treinado: '+cidade+'-'+praia+'- MSE:'+str(loss))

iniciando: CARAGUATATUBA - LAGOA AZUL
Treinando modelo: CARAGUATATUBA-LAGOA AZUL
Modelo Treinado: CARAGUATATUBA-LAGOA AZUL- MSE:3250
iniciando: GUARUJÁ - IPORANGA
Treinando modelo: GUARUJÁ-IPORANGA
